In [1]:

# Choose your model from Hugging Face Hub

# Data must be in the following format:
data = [
    {
        "src": "10 到 15 分钟可以送到吗",
        "mt": "Can I receive my food in 10 to 15 minutes?",
        "ref": "Can it be delivered between 10 to 15 minutes?"
    },
    {
        "src": "Pode ser entregue dentro de 10 a 15 minutos?",
        "mt": "Can you send it for 10 to 15 minutes?",
        "ref": "Can it be delivered between 10 to 15 minutes?"
    }
]
# Call predict method:
# model_output = model.predict(data, batch_size=8, gpus=1)

/opt/miniconda3/envs/crfm-helm2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 30131.49it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/opt/miniconda3/envs/crfm-helm2/lib/python3.9/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: Fals

In [2]:
import sys
import json
import argparse

parser = argparse.ArgumentParser("simple_example")
from helm.benchmark.runner import InstanceGenerations,GenerationSummary
from process_gens import get_gen_summary, get_run_folder, get_gen_summary_from_path

from PostMetric import calculate_post_metric, get_post_metrics
from helm.common.general import ensure_directory_exists, write, asdict_without_nones
import os.path

PyTorch version 2.6.0 available.


In [16]:
# num_beams=32
# model="meta_llama_Llama_3.1_8B_Instruct"
# eval_instances=5
# task_name="wmt_14_language_pair_de_en_"
# run_path=f"snellius_copies/wmt_test"
# metric_names="test"


#first, we want a list of all the examples
#then, we want to create a list of data
#then, we want to add to the stats_dict 

# run_path="./helm_output/sample_return_2/wmt_14_language_pair_de_en_/distilbert_distilgpt2/1_beams/runs/sample_return_2"
run_path="./helm_output/sample_return_16/wmt_14_language_pair_de_en_/distilbert_distilgpt2/1_beams/runs/sample_return_16"

gen_sum_raw_path=f"{run_path}/generation_summary.json"
gen_sum_metric_path=f"{run_path}/generation_summary_metrics.json"
input_path = gen_sum_metric_path if os.path.isfile(gen_sum_metric_path) else gen_sum_raw_path
generation_summary=get_gen_summary_from_path(input_path)


from comet import download_model, load_from_checkpoint

def append_comment_metric(generation_summary):
    model_path = download_model("Unbabel/wmt22-comet-da")
    model = load_from_checkpoint(model_path)
    data = []
    for instance_generation in generation_summary.instance_generations:
        for generated_output in instance_generation.examples:
            data.append({
                "ref" : instance_generation.reference.strip(),
                "src" : instance_generation.prompt.strip(),
                "mt" : generated_output.text.strip()
            })
    model_output = model.predict(data, batch_size=8, gpus=1)

    counter=0
    for instance_generation in generation_summary.instance_generations:
        for generated_output in instance_generation.examples:
            comet_score=model_output.scores[counter]
            generated_output.stats_dict = {} if generated_output.stats_dict is None else generated_output.stats_dict 
            generated_output.stats_dict["comet_score"]=comet_score
            counter+=1

    print(generation_summary.instance_generations[0].examples[0].stats_dict)
    return instance_generation


append_comment_metric(generation_summary)



None


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 19673.10it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/opt/miniconda3/envs/crfm-helm2/lib/python3.9/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
python(69895) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69896) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Predicting DataLoader 0: 100%|██████████| 2/2 [00:02

{'comet_score': 0.2958647906780243}
{'comet_score': 0.2958647906780243}
